In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/5018 Project

/content/drive/.shortcut-targets-by-id/1Yz_k2CASYKkwbh9G60SXjkuhxKPF9kWP/5018 Project


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tensorflow as tf
import transformers
from tensorflow.keras.layers import Input
from transformers import AutoTokenizer
from script.text_cleaning import clean_html, remove_links, remove_special_characters, removeStopWords, remove_, remove_digits, lower, email_address, non_ascii, punct, hashtags

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = TFRobertaModel.from_pretrained('roberta-base')

# Recreate the exact same model, including its weights and the optimizer
# model = tf.keras.models.load_model('/content/drive/MyDrive/5018 Project/saved_model/roberta_model_v2.h5', custom_objects={"TFRobertaModel": transformers.TFRobertaModel})

model = tf.keras.models.load_model('/content/drive/MyDrive/5018 Project/saved_model/bert_base')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from turtle import textinput
import pandas as pd
import numpy as np
# Take input from user
# msg = textinput("Message", "Please enter the email or text message:")
msg = "Congrats! You won a prize! Please click this link www.winnning.com to claim the prize."

def text_preprocess(data,col):
    data[col] = data[col].apply(func=clean_html)
    data[col] = data[col].apply(func=remove_)
    data[col] = data[col].apply(func=removeStopWords)
    data[col] = data[col].apply(func=remove_special_characters)
    data[col] = data[col].apply(func=punct)
    data[col] = data[col].apply(func=non_ascii)
    data[col] = data[col].apply(func=email_address)
    data[col] = data[col].apply(func=lower)
    return data

msg = pd.DataFrame(data=[msg])
cleaned_msg = text_preprocess(msg,0)

print(cleaned_msg[0])

max_len = 50

cleaned_msg = tokenizer(cleaned_msg[0].to_list(),
                   max_length=max_len,
                   padding='max_length',
                   truncation=True,
                   add_special_tokens=True,
                   return_tensors="tf",
                   return_attention_mask=True,
                   return_token_type_ids=True,
                   verbose=True)

# input_ids = Input(shape=(max_len,),dtype=tf.int32,name='input_ids')

test_predictions = model.predict({'input_ids': cleaned_msg['input_ids'], 'attention_mask': cleaned_msg['attention_mask'], 'token_type_ids': cleaned_msg['token_type_ids']})
test_predictions = np.argmax(test_predictions['logits'])

0    congrats you prize please click link www winnn...
Name: 0, dtype: object
1/1 [==============================] - 0s 330ms/step


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pickle
# Load model
with open("/content/drive/MyDrive/5018 Project/saved_model/kmeans_k5.pkl", "rb") as f:
    model_k5 = pickle.load(f)

with open("./saved_model/vectorizer.pk", "rb") as f:
    vectorizer = pickle.load(f)

In [ ]:
if test_predictions == 0:
  print("This is not spam message.")
else:
  y = vectorizer.transform([msg[0][0]])
  pred = model_k5.predict(y)[0]

  cluster_dict = {
      0:"Prize Claim and Winning",
      1:"Mobile Text and Tone Services",
      2:"Email Marketing and Information Lists",
      3:"Holiday and Travel Offers",
      4:"Award and Complimentary Collection"}

  print("This is spam message.")
  print("Type:", cluster_dict[pred])

This is spam message.
Type: Prize Claim and Winning


### Testing the model on YouTube comments

In [ ]:
df_y1 = pd.read_csv("./data/raw/Youtube01-Psy.csv")
df_y2 = pd.read_csv("./data/raw/Youtube02-KatyPerry.csv")
df_y3 = pd.read_csv("./data/raw/Youtube03-LMFAO.csv")
df_y4 = pd.read_csv("./data/raw/Youtube04-Eminem.csv")
df_y5 = pd.read_csv("./data/raw/Youtube05-Shakira.csv")

df_youtube = pd.concat([df_y1, df_y2, df_y3, df_y4, df_y5], ignore_index=False)
df_youtube = df_youtube[['CONTENT','CLASS']]
df_youtube.rename(columns={'CONTENT':'text', 'CLASS':'spam'}, inplace = True)

In [ ]:
cleaned_msg = text_preprocess(df_youtube, 'text')

max_length = 50

cleaned_msg = tokenizer(text=cleaned_msg['text'].to_list(),
                   max_length=max_len,
                   padding='max_length',
                   truncation=True,
                   add_special_tokens=True,
                   return_tensors="tf",
                   return_attention_mask=True,
                   return_token_type_ids=True,
                   verbose=True)


test_predictions = model.predict({'input_ids': cleaned_msg['input_ids'], 'attention_mask': cleaned_msg['attention_mask'], 'token_type_ids': cleaned_msg['token_type_ids']})
test_predictions_binary = np.argmax(test_predictions['logits'], axis = 1)

62/62 [==============================] - 387s 6s/step


In [ ]:
df_youtube['predictions'] = test_predictions_binary

In [ ]:
df_youtube

,text,spam,predictions
0,huh anyway check tube channel kobyoshi02,1,0
1,hey guys check new channel first vid us monkey...,1,0
2,test say murdev com,1,0
3,shaking sexy ass channel enjoy,1,0
4,watch vvtarggvgtwq check,1,0
...,...,...,...
365,love song sing camp time,0,0
366,love song two reasons 1 africa 2 born beautifu...,0,0
367,wow,0,0
368,shakira u wiredo,0,0


In [ ]:
def get_cat(x):
  y = vectorizer.transform([x])
  pred = model_k5.predict(y)[0]

  cluster_dict = {
      0:"Prize Claim and Winning",
      1:"Mobile Text and Tone Services",
      2:"Email Marketing and Information Lists",
      3:"Holiday and Travel Offers",
      4:"Award and Complimentary Collection"}

  return cluster_dict[pred]

In [ ]:
# added this cell to cluster the type of youtube comments
df_youtube['cluster'] = df_youtube['text'].apply(lambda x: get_cat(x))

In [ ]:
df_youtube[df_youtube['spam'] == 1]['cluster'].value_counts()

Email Marketing and Information Lists    999
Mobile Text and Tone Services              4
Holiday and Travel Offers                  2
Name: cluster, dtype: int64

In [ ]:
accuracy = 1 - sum(abs(df_youtube['spam'] - df_youtube['predictions']))/len(df_youtube)
print("Accuracy: ", accuracy)

Accuracy:  0.6221881390593047


In [ ]:
# example of a spam youtube comment categorized

print(df_youtube[df_youtube["predictions"] == 1].head(7))

                                                 text  spam  predictions  \
12                  https twitter com gbphotographygb     1            1   
14  please https premium easypromosapp com voteme ...     1            1   
15  hello gaming art videos scientific experiments...     1            1   
17  www ebay com itm 171183229277 sspagenamestrk m...     1            1   
18               ubuntuone com 40beuutvu2zkxk4utgpz8k     1            1   
19  edm apparel company dedicated bringing music i...     1            1   
21                           subscribe channel people     1            1   

                                  cluster  
12  Email Marketing and Information Lists  
14  Email Marketing and Information Lists  
15  Email Marketing and Information Lists  
17  Email Marketing and Information Lists  
18  Email Marketing and Information Lists  
19  Email Marketing and Information Lists  
21  Email Marketing and Information Lists  
